In [1]:
import sys

sys.path.append(".../pipeline_imaging/imaging")  #'/path/to/your/project'

In [2]:
import pydicom
import imaging_utils
import os

# Folder structures

In [6]:
# Pooled date folders across three sites
# step1 (pooled)/
# └── FLIO/
#  ├── UW_FLIO_20240730-20240827/
#  └── UW_FLIO_20240730-20240827/
#      ├──457_AIREADI_1375
#      └──268_AIREADI_4248


# => flio_instance.organize =>

# Files orangnized by per scan
# step2/
# └── FLIO/
#  ├── flio_268_AIREADI_4248_2024_07_01-13_00_29_OD
#  ├── flio_268_AIREADI_4248_2024_07_01-13_03_01_OS
#  ├── ...
#  └── flio_459_AIREADI_1376_2024_07_25-13_02_15_OS
#      ├── AF_Intensity_LONG-Channel.bmp
#      ├── AF_Intensity_Long+Short_Channels.bmp
#      ├── AF_Intensity_Short-Channel.bmp
#      ├── measurement_info.html
#      └── Measurement.sdt

#  => spectralis_instance.convert1 =>

# From a scan, dicom files are created
# step3/
# └── FLIO/
#     ├── 1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     ├── 1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     └── ... (additional DICOM files)


#  => flio_instance.converted2 =>

# From a dicom file, NEMA compliant DICOM files are created
# step4/
# └── FLIO/
#     ├── converted_1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     ├── converted_1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     └── ... (additional DICOM files)

#  => imaging_utils.format_file =>
#  => flio_instance.metadata =>

# De-identified again, file renamed, and organized in a final structure for data relase
# step5/
# └── FLIO/
#    ├── retinal_flio/
#         └── heidelberg_flio/
#               ├── 1001
#               ├── 1002
#               ├── ... (additional folders)
#               ├── 1003
#                     └── 1375_flio_long_wavelength_r_1.2.826.0.1.3680043.8.498.12481560985877485897830542137951711344.dcm
#                     └── 1375_flio_long_wavelength_r_1.2.826.0.1.3680043.8.498.12481560985877485897830542137951711344.dcm

# step_metadata/
# └── FLIO/
#    ├── retinal_flio/
#         ├── filename1.json
#         ├── filename2.json
#         ├── filename3.json
#         └── ... (additional JSON files)

# FLIO Instance

In [ ]:
from imaging_flio_root import Flio

flio_instance = Flio()

In [6]:
device = "Flio"

main = f"/Volume"

step1 = f"{main}"
step2 = f"{main}/step2"
step3 = f"{main}/step3"
step4 = f"{main}/step4"
step5 = f"{main}/step5"
step_metadata = f"{main}/metadata"

jsonpath = ".../pipeline_imaging/imaging/flio_uid_data.json"


# Organize by scan

In [7]:
folders = imaging_utils.list_subfolders(f"{step1}/{device}")
for folder in folders:
    organize_result = flio_instance.organize(folder, f"{step2}/{device}")

# Convert1 - convert a folder (worth one scan) to two dicom files

In [ ]:
folders = imaging_utils.list_subfolders(f"{step2}/{device}")

for folder in folders:
    if "flio" in folder:
        flio_instance.convert1(folder, f"{step3}/{device}", jsonpath)

# Convert 2 - convert dicom files to NEMA compliant dicom files

In [9]:
filtered_list = imaging_utils.get_filtered_file_names(f"{step3}/{device}")

for file in filtered_list:
    if "flio" in file:
        cov_result = flio_instance.convert2(file, f"{step4}/{device}")

# Files are de-identified again, file renamed, and organized in a final structure for data relase + output metadata

In [12]:
filelist = imaging_utils.get_filtered_file_names(f"{step4}/{device}")

for file in filelist:
    if "flio" in file:
        full_file_path = imaging_utils.format_file(file, f"{step5}/{device}")
        flio_instance.metadata(full_file_path, f"{step_metadata}/{device}")